In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

## Indexing con Xarray


Una vez tengamos nuestro data array queremos "aprovechar" nuestras coordenas, y por algun motivo no perder la posibilidad de ser infelices:
Para ello podemos aprovecharlos y seleccionar a traves indices (porque wtf?) o a traves de nombres dimension:


In [30]:
example = xr.DataArray(
    np.arange(0, 60).reshape(3, 4, 5),
    coords={
        "dim1": ["A", "B", "C"],
        "dim2": ["a", "b", "c", "d"],
        "dim3": np.arange(0, 5)
    },
    dims=("dim1", "dim2", "dim3")
)

In [26]:
example[2,1,2]

<xarray.DataArray ()> Size: 8B
array(47)
Coordinates:
    dim1     <U1 4B 'C'
    dim2     <U1 4B 'b'
    dim3     int64 8B 2

In [24]:
example.loc["C","b",2]

<xarray.DataArray ()> Size: 8B
array(47)
Coordinates:
    dim1     <U1 4B 'C'
    dim2     <U1 4B 'b'
    dim3     int64 8B 2

In [19]:
example.sel(dim1 = slice("A","C"))

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 5)> Size: 480B
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])
Coordinates:
  * dim1     (dim1) <U1 12B 'A' 'B' 'C'
  * dim2     (dim2) <U1 16B 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 40B 0 1 2 3 4

In [ ]:
example.isel(dim1 = 0)

In [33]:
example.drop_sel(dim3 = [2,3])

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 3)> Size: 288B
array([[[ 0,  1,  4],
        [ 5,  6,  9],
        [10, 11, 14],
        [15, 16, 19]],

       [[20, 21, 24],
        [25, 26, 29],
        [30, 31, 34],
        [35, 36, 39]],

       [[40, 41, 44],
        [45, 46, 49],
        [50, 51, 54],
        [55, 56, 59]]])
Coordinates:
  * dim1     (dim1) <U1 12B 'A' 'B' 'C'
  * dim2     (dim2) <U1 16B 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 24B 0 1 4

In [ ]:
example2 = example.assign_coords(dim1 = [97.8,98.2,344.23123])
example2

In [ ]:
example2.sel(dim1 = 97)

# DateTime Indexing:

Podes tener una dimension que sus coordenas sean fechas, entonces logicamente deberia andar bien el seleccionar una fecha puntual.. anda bien :D, pero no solo eso sino que dado que esta guardado en formato fecha podes hacer cosas como:
| Objetivo                                                            | Código `xarray`                                                             | Tipo             |
| ------------------------------------------------------------------- | --------------------------------------------------------------------------- | ---------------- |
| Fecha exacta (día y hora)                                           | `ds.sel(time='2013-01-01 06:00')`                                           | Exacta           |
| Día completo (todas las horas)                                      | `ds.sel(time='2013-01-01')`                                                 | Exacta           |
| Mes completo (todas las fechas de ese mes y año)                    | `ds.sel(time='2014-05')`                                                    | Exacta           |
| Año completo                                                        | `ds.sel(time='2013')`                                                       | Exacta           |
| Slice entre dos fechas                                              | `ds.sel(time=slice('2013-01-01', '2013-01-10'))`                            | Slice            |
| Slice entre dos fechas con hora                                     | `ds.sel(time=slice('2013-01-01 03:00', '2013-01-02 12:00'))`                | Slice            |
| Slice de mes a mes                                                  | `ds.sel(time=slice('2013-01', '2013-03'))`                                  | Slice            |
| Slice de año a año                                                  | `ds.sel(time=slice('2012', '2014'))`                                        | Slice            |
| Todos los valores de un **mes** sin importar año                    | `ds.sel(time=ds.time.dt.month == 7)`                                        | Condicional      |
| Todos los valores de un **día del mes** sin importar mes/año        | `ds.sel(time=ds.time.dt.day == 15)`                                         | Condicional      |
| Todos los valores de un **año específico**                          | `ds.sel(time=ds.time.dt.year == 2014)`                                      | Condicional      |
| Todos los valores de una **hora específica**                        | `ds.sel(time=ds.time.dt.hour == 6)`                                         | Condicional      |
| Todos los valores de un **día de la semana (0=lunes)**              | `ds.sel(time=ds.time.dt.dayofweek == 0)`                                    | Condicional      |
| Todos los valores de una **fecha específica combinada (mes + día)** | `ds.sel(time=(ds.time.dt.month == 12) & (ds.time.dt.day == 24))`            | Combinada        |
| Estación: Verano (dic-ene-feb)                                      | `ds.sel(time=ds.time.dt.month.isin([12, 1, 2]))`                            | Estacional       |
| Estación: Otoño (mar-abr-may)                                       | `ds.sel(time=ds.time.dt.month.isin([3, 4, 5]))`                             | Estacional       |
| Estación: Invierno (jun-jul-ago)                                    | `ds.sel(time=ds.time.dt.month.isin([6, 7, 8]))`                             | Estacional       |
| Estación: Primavera (sep-oct-nov)                                   | `ds.sel(time=ds.time.dt.month.isin([9, 10, 11]))`                           | Estacional       |
| Estación combinada con año (ej: invierno 2013)                      | `ds.sel(time=(ds.time.dt.year == 2013) & ds.time.dt.month.isin([6, 7, 8]))` | Estacional + Año |
| Días impares                                                        | `ds.sel(time=ds.time.dt.day % 2 == 1)`                                      | Personalizada    |
| Último día del mes                                                  | `ds.sel(time=ds.time.dt.is_month_end)`                                      | Especial         |
| Primer día del mes                                                  | `ds.sel(time=ds.time.dt.is_month_start)`                                    | Especial         |


# numpy diferencias

A diferencia de los arrays en numpy, el indexing en xarray es ortogonal (mientras en numpy es puntual)

In [123]:
np_array = np.arange(1, 26).reshape(5, 5)
da = xr.DataArray(np_array, dims=["x", "y"])

In [124]:
np_array[[0, 2, 4], [0, 2, 4]]

array([ 1, 13, 25])

In [125]:
da[[0, 2, 4], [0, 2, 4]]

<xarray.DataArray (x: 3, y: 3)> Size: 72B
array([[ 1,  3,  5],
       [11, 13, 15],
       [21, 23, 25]])
Dimensions without coordinates: x, y

<img src="images/indexing.png" width="600" height="600">

In [131]:
indices = xr.DataArray([0,2,4], dims = "points")
da.isel(x = indices , y = indices) 

<xarray.DataArray (points: 3)> Size: 24B
array([ 1, 13, 25])
Dimensions without coordinates: points

# Conditional indexing and masking
Queremos aprovechar condiciones booleanas para nuestro indice, podemos devolver el mismo dataArray con aquellas posiciones que cumplen nuestra condicion booleana modificado o devolver solo aquellas que cumplen nuestra condicion booleana

In [171]:
data = xr.DataArray(np.arange(0,100).reshape(10,10), dims = ('x','y'), coords = {'x':np.arange(0,10), 'y':np.arange(0,10)})
data

<xarray.DataArray (x: 10, y: 10)> Size: 800B
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])
Coordinates:
  * x        (x) int64 80B 0 1 2 3 4 5 6 7 8 9
  * y        (y) int64 80B 0 1 2 3 4 5 6 7 8 9

In [159]:
data.where(data<50, 99-data)

<xarray.DataArray (x: 10, y: 10)> Size: 800B
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [49, 48, 47, 46, 45, 44, 43, 42, 41, 40],
       [39, 38, 37, 36, 35, 34, 33, 32, 31, 30],
       [29, 28, 27, 26, 25, 24, 23, 22, 21, 20],
       [19, 18, 17, 16, 15, 14, 13, 12, 11, 10],
       [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0]])
Coordinates:
  * x        (x) int64 80B 0 1 2 3 4 5 6 7 8 9
  * y        (y) int64 80B 0 1 2 3 4 5 6 7 8 9

In [160]:
xr.where((data.x + data.y) % 2 == 0, 0,data)

<xarray.DataArray (x: 10, y: 10)> Size: 800B
array([[ 0,  1,  0,  3,  0,  5,  0,  7,  0,  9],
       [10,  0, 12,  0, 14,  0, 16,  0, 18,  0],
       [ 0, 21,  0, 23,  0, 25,  0, 27,  0, 29],
       [30,  0, 32,  0, 34,  0, 36,  0, 38,  0],
       [ 0, 41,  0, 43,  0, 45,  0, 47,  0, 49],
       [50,  0, 52,  0, 54,  0, 56,  0, 58,  0],
       [ 0, 61,  0, 63,  0, 65,  0, 67,  0, 69],
       [70,  0, 72,  0, 74,  0, 76,  0, 78,  0],
       [ 0, 81,  0, 83,  0, 85,  0, 87,  0, 89],
       [90,  0, 92,  0, 94,  0, 96,  0, 98,  0]])
Coordinates:
  * x        (x) int64 80B 0 1 2 3 4 5 6 7 8 9
  * y        (y) int64 80B 0 1 2 3 4 5 6 7 8 9

In [176]:
arr = np.random.randint(0,10,data.shape)
data = data.assign_coords(mask = (data.dims,arr))
data.where(data.mask == 0,drop = True)

<xarray.DataArray (x: 6, y: 6)> Size: 288B
array([[nan, nan, 32., nan, nan, nan],
       [nan, nan, 42., nan, nan, nan],
       [nan, nan, 52., nan, nan, nan],
       [nan, nan, 62., 65., nan, 68.],
       [70., 71., nan, nan, nan, nan],
       [nan, nan, nan, nan, 87., nan]])
Coordinates:
  * x        (x) int64 48B 3 4 5 6 7 8
  * y        (y) int64 48B 0 1 2 5 7 8
    mask     (x, y) int64 288B 8 7 0 2 9 1 5 9 0 2 3 8 ... 0 7 1 5 4 5 4 8 3 0 3

## COMPUTATION
Probablemente querramos hacer cosas mas interesantes que preguntar por datos en nuestro xarray, asi que veamos que podemos hacer :D
# BASIC
podemos aplicar funciones y hacer cualquier reduccion estandar de numpy en las coordenadas (mean, min, max, sum, etc.)

In [38]:
example = xr.DataArray(
    np.zeros((3,4)),
    coords = {
        "x" : np.arange(0,3),
        "y" : np.arange(0,4)
    },
    dims = ("x","y")
)
example

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [39]:
def sumar(a,b):
    return a+b

xr.apply_ufunc(sumar,example,2)

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [42]:
from math import *
(example + 2) ** 2

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [53]:
arr = np.array([
    [0, 1, 2, 3],
    [8, 9, 10, 11],
    [0, 5, 6, 7]
])
example.data = arr
example

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[ 0,  1,  2,  3],
       [ 9,  9, 10, 11],
       [ 0,  5,  6,  7]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [55]:
example.mean(dim = "x")

<xarray.DataArray (y: 4)> Size: 32B
array([3., 5., 6., 7.])
Coordinates:
  * y        (y) int64 32B 0 1 2 3

## Computing with multiple objects
La aplicacion de funciones entre multiples objetos esta claramente definida entre DataArrays que tienen las mismas dimensiones y las mismas longitudes entre estas dimensiones, pues por ejemplo una suma entre dos DataArray daria como resultado la suma par a par de coordenadas iguales. En general es comparable con numpy y cualquier operacion que hacer entre las numpy arrays se puede hacer entre Data Arrays 
Pero que sucede cuando tenemos Data Arrays de diferentes dimensiones? o diferentes longitudes en las dimensiones?
Estrategias de broadcasting y Alignment, se pueden realizar en numpy pero no estan integradas

# Broadcasting:
Si quisiera sumar 2 array 1-dimensionales, a = [1,2] b = [3,4] pero las dimensiones son x e y respectivamente, entonces xarray los interpreta como que tienen diferentes dimensiones y tiene que llevar ambos a la cantidad de dimensiones correctas. Para ello en cada array: copia, en toda la direccion que no tiene el array, los datos en las dimensiones que si.
Por lo que nos quedaria a = [[1,1],[2,2]] b = [[3,4],[3,4]]

# Alignment:
Las dimensiones son las mismas, pero las coordenadas no. Entonces xarray debe poder determinar un resultado, mas particularmente un tamaño de las coordenadas del resultado. join: inner, left, right, outer

In [58]:
a = xr.DataArray(np.arange(1,3), dims = "x")
b = xr.DataArray(np.arange(3,5), dims = "y")
a+b

<xarray.DataArray (x: 2, y: 2)> Size: 32B
array([[4, 5],
       [5, 6]])
Dimensions without coordinates: x, y

In [59]:
xr.broadcast(a,b)

(<xarray.DataArray (x: 2, y: 2)> Size: 32B
 array([[1, 1],
        [2, 2]])
 Dimensions without coordinates: x, y,
 <xarray.DataArray (x: 2, y: 2)> Size: 32B
 array([[3, 4],
        [3, 4]])
 Dimensions without coordinates: x, y)

In [61]:
arr1 = xr.DataArray(
    np.arange(12).reshape(3, 4),
    dims=("space", "time"),
    coords={"space": ["a", "b", "c"], "time": [0, 1, 2, 3]},
)
arr2 = xr.DataArray(
    np.arange(14).reshape(2, 7),
    dims=("space", "time"),
    coords={"space": ["b", "d"], "time": [-2, -1, 0, 1, 2, 3, 4]},
)
display(arr1)
display(arr2)

<xarray.DataArray (space: 3, time: 4)> Size: 96B
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])
Coordinates:
  * space    (space) <U1 12B 'a' 'b' 'c'
  * time     (time) int64 32B 0 1 2 3

<xarray.DataArray (space: 2, time: 7)> Size: 112B
array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13]])
Coordinates:
  * space    (space) <U1 8B 'b' 'd'
  * time     (time) int64 56B -2 -1 0 1 2 3 4

<img src="images/alignment.png" width="600" height="600">

In [70]:
xr.align(a,b)

(<xarray.DataArray (x: 2)> Size: 16B
 array([1, 2])
 Dimensions without coordinates: x,
 <xarray.DataArray (y: 2)> Size: 16B
 array([3, 4])
 Dimensions without coordinates: y)

In [71]:
with xr.set_options(arithmetic_join="outer"):
    result = arr1 + arr2
result

<xarray.DataArray (space: 4, time: 7)> Size: 224B
array([[nan, nan, nan, nan, nan, nan, nan],
       [nan, nan,  6.,  8., 10., 12., nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan]])
Coordinates:
  * space    (space) <U1 16B 'a' 'b' 'c' 'd'
  * time     (time) int64 56B -2 -1 0 1 2 3 4

# Agrupemos info
Queremos agrupar datos. sel e isel filtran, pero que pasa si quisieramos relacionar todos los datos de la misma hora por ejemplo?
Agrupar por:
- Groupby
- resample
- rolling
- coarsen

In [73]:
import pandas as pd

In [77]:
data = xr.DataArray(
    np.arange(0,36),
    dims = ("time"),
    coords = {"time" : pd.date_range("2020-01-01", periods = 36, freq = "2h" )}
)
data

<xarray.DataArray (time: 36)> Size: 288B
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35])
Coordinates:
  * time     (time) datetime64[ns] 288B 2020-01-01 ... 2020-01-03T22:00:00

In [79]:
data.groupby("time.hour").mean()

<xarray.DataArray (hour: 12)> Size: 96B
array([12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])
Coordinates:
  * hour     (hour) int64 96B 0 2 4 6 8 10 12 14 16 18 20 22

In [98]:
def remove_time_mean(x):
    return x - x.mean(dim="time")


data = data.groupby("time.hour").map(remove_time_mean)
data

<xarray.DataArray (time: 36)> Size: 288B
array([-12., -12., -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,
        12.,  12.,  12.])
Coordinates:
  * time     (time) datetime64[ns] 288B 2020-01-01 ... 2020-01-03T22:00:00

In [100]:
data.resample(time="2h").sum()

<xarray.DataArray (time: 36)> Size: 288B
array([-12., -12., -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,
        12.,  12.,  12.])
Coordinates:
  * time     (time) datetime64[ns] 288B 2020-01-01 ... 2020-01-03T22:00:00

In [104]:
data.rolling(time=3).mean()

<xarray.DataArray (time: 36)> Size: 288B
array([ nan,  nan, -12., -12., -12., -12., -12., -12., -12., -12., -12.,
       -12.,  -8.,  -4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   4.,   8.,  12.,  12.,  12.,  12.,  12.,  12.,  12.,
        12.,  12.,  12.])
Coordinates:
  * time     (time) datetime64[ns] 288B 2020-01-01 ... 2020-01-03T22:00:00

In [103]:
data.coarsen(time = 2).mean()

<xarray.DataArray (time: 18)> Size: 144B
array([-12., -12., -12., -12., -12., -12.,   0.,   0.,   0.,   0.,   0.,
         0.,  12.,  12.,  12.,  12.,  12.,  12.])
Coordinates:
  * time     (time) datetime64[ns] 144B 2020-01-01T01:00:00 ... 2020-01-03T21...

# 📊 Técnicas de agrupamiento en xarray

| Técnica    | Agrupa por...           | Requiere coord. especial   | Tipo de ventana        | Forma de salida        | Comentario clave                              |
| ---------- | ----------------------- | -------------------------- | ---------------------- | ---------------------- | --------------------------------------------- |
| `groupby`  | Etiquetas (categorías)  | Categórica o derivada      | No                     | Reducida o expandida   | Agrupa según los valores únicos de una coord. |
| `resample` | Frecuencias temporales  | Coordenada tipo `datetime` | No (bloques por fecha) | Reducida               | Agrupa por fechas como '1D', '1M', etc.       |
| `rolling`  | Tamaño fijo deslizante  | Ninguna                    | Solapada               | Misma forma (con NaNs) | Ventana móvil como media móvil                |
| `coarsen`  | Tamaño fijo no solapado | Ninguna                    | No solapada            | Reducida               | Agrupa en bloques fijos (e.g., cada 3)        |


# Weighter

In [116]:
import xarray as xr
import numpy as np

data = xr.DataArray(
    np.arange(12).reshape(3, 4),
    dims=("lat", "lon"),
    coords={"lat": [10, 20, 30], "lon": [100, 110, 120, 130]},
    name="temperatura"
)

pesos = xr.DataArray([0.2, 0.3, 0.5], dims="lat")

media_ponderada = data.weighted(pesos).mean(dim="lat")

media_ponderada

<xarray.DataArray 'temperatura' (lon: 4)> Size: 32B
array([5.2, 6.2, 7.2, 8.2])
Coordinates:
  * lon      (lon) int64 32B 100 110 120 130